In [2]:
!pip install transformers
!pip install ipywidgets
!pip install IProgress

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 4.7 MB 22.1 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 88.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 90.6 MB/s eta 0:00:01
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 134 kB 22.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 93.7 MB/s eta 0:00:01
     |████████████████████████████████| 384 kB 86.5 MB/s eta 0:00:01
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import numpy as np

In [4]:
# imports
# dataset and dataloaders
# get span representation function
# custom model class
# custom training loop
# instantiate model
# train for some epochs
# put model in eval
# get predictions
# classification report

In [5]:
paragraph = """

Some people may argue that children will be more material,
neglect their study for earning money or be exploited by the
employers. However, if children get good care and
instructions from their parents, they can take advantages of the
work to learn valuable things and avoid going in a wrong way

"""

In [6]:
# define spans

am_1_span = (0, 4)
ac_1_span = (5, 22)

am_2_span = (23, 24)
ac_2_span = (25, 54)

In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [65]:
tokenized_input = tokenizer(paragraph, return_tensors="pt")

In [66]:
tokenized_input['input_ids'].shape[1]

57

In [67]:
tokenized_input['input_ids']

tensor([[  101,  2070,  2111,  2089,  7475,  2008,  2336,  2097,  2022,  2062,
          3430,  1010, 19046,  2037,  2817,  2005,  7414,  2769,  2030,  2022,
         18516,  2011,  1996, 12433,  1012,  2174,  1010,  2065,  2336,  2131,
          2204,  2729,  1998,  8128,  2013,  2037,  3008,  1010,  2027,  2064,
          2202, 12637,  1997,  1996,  2147,  2000,  4553,  7070,  2477,  1998,
          4468,  2183,  1999,  1037,  3308,  2126,   102]])

In [68]:
# ok, so without CLS and SEP, we have input_ids of len 55. exactly as the paragraph length. correct.

In [69]:
# but we can separate the AMs and ACs before the tokenization. which is the better way? 
# for now, do it after the tokenization and then discuss with Mr Cabessa.

In [70]:
# offset the tokizer input_ids to ignore the CLS token

In [71]:
tokenized_input['input_ids'] = tokenized_input['input_ids'].index_select(1, torch.tensor(list(range(1, tokenized_input['input_ids'].shape[1] - 1))))

In [72]:
tokenized_input['input_ids']

tensor([[ 2070,  2111,  2089,  7475,  2008,  2336,  2097,  2022,  2062,  3430,
          1010, 19046,  2037,  2817,  2005,  7414,  2769,  2030,  2022, 18516,
          2011,  1996, 12433,  1012,  2174,  1010,  2065,  2336,  2131,  2204,
          2729,  1998,  8128,  2013,  2037,  3008,  1010,  2027,  2064,  2202,
         12637,  1997,  1996,  2147,  2000,  4553,  7070,  2477,  1998,  4468,
          2183,  1999,  1037,  3308,  2126]])

In [73]:
# ok.

In [74]:
# perhaps I should do a function will will do all this data processing, 

In [75]:
# so now get the tokenized inputs of the various spans

In [82]:
am_1_tokenized_span = tokenized_input['input_ids'].index_select(1, torch.tensor(list(range(am_1_span[0], am_1_span[1] + 1))))

In [83]:
am_1_tokenized_span

tensor([[2070, 2111, 2089, 7475, 2008]])

In [84]:
ac_1_tokenized_span = tokenized_input['input_ids'].index_select(1, torch.tensor(list(range(ac_1_span[0], ac_1_span[1] + 1))))

In [85]:
am_2_tokenized_span = tokenized_input['input_ids'].index_select(1, torch.tensor(list(range(am_2_span[0], am_2_span[1] + 1))))

In [86]:
ac_2_tokenized_span = tokenized_input['input_ids'].index_select(1, torch.tensor(list(range(ac_2_span[0], ac_2_span[1] + 1))))

In [87]:
ac_2_tokenized_span

tensor([[ 1010,  2065,  2336,  2131,  2204,  2729,  1998,  8128,  2013,  2037,
          3008,  1010,  2027,  2064,  2202, 12637,  1997,  1996,  2147,  2000,
          4553,  7070,  2477,  1998,  4468,  2183,  1999,  1037,  3308,  2126]])

In [ ]:
# now these should be given to the custom BERT class

In [90]:
tokenized_spans = [am_1_tokenized_span, am_2_tokenized_span, ac_1_tokenized_span, ac_2_tokenized_span]

In [ ]:
# this is just an example object. when properly done, this should include ALL tokenized spans in of all AMs/ACs in the paragraph.
# this object should be returned by a big tokenization function

In [91]:
tokenized_spans

[tensor([[2070, 2111, 2089, 7475, 2008]]),
 tensor([[1012, 2174]]),
 tensor([[ 2336,  2097,  2022,  2062,  3430,  1010, 19046,  2037,  2817,  2005,
           7414,  2769,  2030,  2022, 18516,  2011,  1996, 12433]]),
 tensor([[ 1010,  2065,  2336,  2131,  2204,  2729,  1998,  8128,  2013,  2037,
           3008,  1010,  2027,  2064,  2202, 12637,  1997,  1996,  2147,  2000,
           4553,  7070,  2477,  1998,  4468,  2183,  1999,  1037,  3308,  2126]])]

In [ ]:
class customBERTKuri(torch.nn.Module):
    
    def __init__(self, model_am, model_ac, model_combined, tokenized_spans, nb_classes):

        super(customBERTKuri, self).__init__()
        

        self.model_am = model_am
        self.model_ac = model_ac
        self.model_combined = model_combined
        
        # don't need embedding anymore because we will use BERT embeddings.
        
        # self.embedding = embedding
        
        # self.embedding_dim = embedding.word_embeddings.embedding_dim
        
        self.nb_classes = nb_classes

        self.linear = torch.nn.Linear(self.embedding_dim, self.nb_classes)


    def forward(self, x):
  
        # 1. Embedding 
  
        # para_embedding = embedding(paragraph)
    
        # 2. AM bert model
        
        am_model_output = self.model_am(tokenized_spans['markers'])
        
        # let's say tokenized_spans would be an object where we can access AM spans with 'markers' key
        # similarly for AC spans with 'components' key
        
        ac_model_output = self.model_ac(tokenized_spans['components'])
        
            
        # 3. Get BERT minus span representations / pooling
        
        span_representation_am = get_span_representation(am_model_output)
        span_representation_ac = get_span_representation(ac_model_output)
        
        # span_representation_am consists of vectors of shape nr_ams x 768 (or whatever bert hidden dim we set). in this example: 2 x 768
        # similarly for span_representation_ac
        
        
        # 4. concatenate them for the combined model
        # one by one
        
        
        bert_minus_adu_representations = get_concatenated_adu_representations(span_representation_am, span_representation_ac)
        
        # bert_minus_adu_representations will consists ordered concatenations of AMs and ACs.
        # will be of 2 * shape nr_adus in para x 768 (or whatever bert hidden dim we set). in this example: 4 x 768
        
        
        # 5. contextualize them in the combined model
        
        combined_model_output = self.model_combined(bert_minus_adu_representations)        
                    
        # 6. FC layer
        
        outputs = self.linear(combined_model_output)

        return outputs

In [ ]:
def train(model, criterion, optimizer, train_dataloader, epochs=1, iter_steps=100):


    loss_l = []
    n_iter = 0

    #  loop over epochs
    for epoch in range(int(epochs)):

        # loop over batches
        for _, batch in enumerate(train_dataloader):

            outputs = model(batch["text"])               # outputs
            labels = batch["label"].type(torch.int64)    # labels (converted to int for the loss)

            loss = criterion(outputs, labels)         # compute loss
            optimizer.zero_grad()                     # reset optimizer gradient
            loss.backward()                           # backward pass
            optimizer.step()                          # gradient update

            n_iter += 1

            if n_iter % iter_steps == 0:
                print("Iteration: {iteration} Loss: {loss}".format(iteration=n_iter, loss=loss.item()))
                loss_l.append(loss.item())

    print("Training complete.")

    return loss_l

In [ ]:
def get_span_representation(PARAGRAPH, SPAN_START, SPAN_END):
    
    tokenized_input = tokenizer(PARAGRAPH, return_tensors="pt")
    outputs = model(**tokenized_input, output_hidden_states=True)
    
    hidden_states = outputs[1] #  outputs[1] instead of [2], because we have no loss now, because we passed no labels
    
    
    # kuri-minus = h[j] - h[i-1]; h[i] - h[j+1]; h[i-1]; h[j+1]
    
    # first_term
    
    span_end_tensor = outputs[1][0][0][SPAN_END] # outputs[1] instead of [2], because we have no loss now, because we passed no labels
    span_start_tensor = outputs[1][0][0][SPAN_START - 1] # outputs[1] instead of [2], because we have no loss now, because we passed no labels
    span_first_term_tensor = span_end_tensor - span_start_tensor
    
    # second_term 
    
    span_start_tensor_2 = outputs[1][0][0][SPAN_START] # outputs[1] instead of [2], because we have no loss now, because we passed no labels
    span_end_tensor_2 = outputs[1][0][0][SPAN_END + 1] # outputs[1] instead of [2], because we have no loss now, because we passed no labels
    span_second_term_tensor = span_start_tensor_2 - span_end_tensor_2
    
    # third_term 
    
    span_tensor_3 = outputs[1][0][0][SPAN_START - 1] # outputs[1] instead of [2], because we have no loss now, because we passed no labels
    
    
    # fourth_term 
    
    span_tensor_4 = outputs[1][0][0][SPAN_END + 1] # outputs[1] instead of [2], because we have no loss now, because we passed no labels
    
    span_minus_tensor = torch.cat((span_first_term_tensor, span_second_term_tensor, span_tensor_3, span_tensor_4))
    
    return span_minus_tensor

In [92]:
def get_concatenated_adu_representations(span_representation_am, span_representation_ac):
    
    adu_representations = []
    
    for i in range(len(span_representation_am)):
        
        adu_representation = torch.concat(span_representation_am[i], span_representation_ac[i])
        adu_representations.append(adu_representation)
        
    return adu_representations